## 初始化Api

In [ ]:
import json

import requests

url = "http://energyscene.zenergyai.com:38080/api/"
payload = json.dumps({"username": "zenergy@zenergy.ai", "password": "ZEnergy.Ai2312"})
response = requests.post(f"{url}auth/login", data=payload)
token = response.json()["token"]

## 时间戳转换

In [ ]:
import time


def get_datetime_stamp(datetime_str):
    """
    日期时间字符串转换为时间戳
    :param datetime_str: 日期时间字符串
    :return: 时间戳
    """
    time_array = time.strptime(datetime_str, "%Y-%m-%d %H:%M:%S")
    time_stamp = int(time.mktime(time_array)) * 1000  # 转换为毫秒
    return time_stamp

## 导入设备Id

In [ ]:
import pandas as pd

device_df = pd.read_csv('../data/ThingsBoardDevice.csv')
device_df.set_index('ProjectId', inplace=True)
device_df

## 获取数据

In [ ]:
# 请求参数
device_id = device_df.loc['天辰1']['Eb-DeviceId']
keys = "chg,dhg,idle,err"
start_ts = get_datetime_stamp("2024-02-02 22:00:00")
# end_ts = get_datetime_stamp("2024-01-26 3:00:00")
end_ts = int(time.time()) * 1000 # now
# 发出请求
response = requests.get(
    f"{url}plugins/telemetry/DEVICE/{device_id}/values/timeseries",
    headers={"X-Authorization": f"Bearer {token}"},
    params={"keys": keys.replace(" ", ""),
            "startTs": start_ts,
            "endTs": end_ts,
            "interval": 60000, 
            "limit": 50000, 
            "agg": "NONE"})
json = response.json()

## 处理原始数据

In [ ]:
from functools import reduce

# dic列表
dfs = []
for key, value in json.items():
    df = pd.DataFrame(value)
    df['value'] = df['value'].astype(float)  # value列转换类型
    df.rename(columns={'value': key}, inplace=True)  # 重命名列名
    dfs.append(df)  # 添加到列表
# 合并数据
df = reduce(lambda x, y: pd.merge(x, y, on="ts", how="outer"), dfs)
df['ts'] = pd.to_datetime(df['ts'], unit='ms', utc=True).dt.tz_convert('Asia/Shanghai')  # 转换时间戳
df.set_index('ts', inplace=True)  # 设置索引
df.sort_index(inplace=True)  # 按时间排序
# df.ffill(inplace=True)  # 前向填充
# 输出数据
df

## 绘制图表

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.update_layout(title_text=keys)
fig.update_xaxes(title_text='时间')
for column in df.columns:
    fig.add_trace(go.Scatter(x=df.index, y=df[column], mode='lines+markers', name=column))
fig.show()

In [ ]:
import plotly.express as px

px.box(df, points="all")
# px.histogram(df, nbins=100)